In [ ]:
import config
from langgraph.graph import START, END, StateGraph, add_messages
from typing_extensions import TypedDict
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.messages import AIMessage, HumanMessage, BaseMessage
from collections.abc import Sequence
from typing import Literal, Annotated

Get Familiar with the add_messgage Function


In [ ]:
my_list = add_messages(
    [HumanMessage('hi , im xavier castillo'),
    AIMessage("Hey, Xavier, how can I help you?")],
    
    [HumanMessage('Could you summarize todays news?')]
    
)

In [ ]:
my_list

Define the state
by default, State values are replaced after each node.

In [ ]:
class State(TypedDict):
    message: Annotated[Sequence[BaseMessage], add_messages] #Annotated lets us attach meta data to a type

In [ ]:
chat = ChatOpenAI(
    model='gpt-4o',
    api_key=config.api_key,
    seed = 365,
    temperature = 0,
    max_completion_tokens=100
)

In [ ]:
def ask_question(state: State) -> State:
    print("\n----> Entering ask question:")
    for m in state.get("messages", []):
        m.pretty_print()

    question = "What is your question?"
    print(question)

    user_text = input()

    # ✅ conserva historial + agrega 2 mensajes nuevos
    return {
        "messages": state.get("messages", []) + [
            AIMessage(content=question),
            HumanMessage(content=user_text),
        ]
    }

In [ ]:
def ask_another_question(state: State)->State:
    print(f"\n -----> Entering ask another question: ")
    for i in state['messages']:
        i.pretty_print()

    question = 'would you like to ask one more questin? (yes / no )'
    print(question)
    return State(messages = [AIMessage(question), HumanMessage(input())] )

In [ ]:
def routing_function(state: State) -> Literal["ask_question", "__end__"]:
    
    if state["messages"][-1].content == "yes":
        return "ask_question"
    else:
        return "__end__"

In [ ]:
def chatbot(state:State)->State:
    print(f"\n----> Entering chatbot:")
    response = chat.invoke(state["messages"])
    response.pretty_print()

    return State(messages=[response])

In [ ]:
graph = StateGraph(State)

In [ ]:
graph.add_node("ask_question", ask_question)
graph.add_node("chatbot", chatbot)
graph.add_node("ask_another_question", ask_another_question)

graph.add_edge(START, "ask_question")
graph.add_edge("ask_question", "chatbot")
graph.add_edge("chatbot", "ask_another_question")
graph.add_conditional_edges(source = "ask_another_question", 
                            path = routing_function)

In [ ]:
graph_compile = graph.compile()

In [ ]:
graph_compile

In [ ]:
graph_compile.invoke(State(messages=[]))